# Run the SONAR-netCDF4 v1 convention checker on an EK80 file

7/2/2023

In [1]:
import echopype as ep
from echopype.testing import TEST_DATA_FOLDER

from sonarnetcdf4_echopype_checker import cc

ep.__version__

'0.6.4.dev132+g439819b3.d20230609'

## Convert the target raw file to an EchoData object

In [2]:
ed = ep.open_raw(TEST_DATA_FOLDER / "ek80/2019118 group2survey-D20191214-T081342.raw", sonar_model='EK80')

# Other, contrasting EK80 files
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek80_bb_with_calibration/2018115-D20181213-T094600.raw", sonar_model='EK80')
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek80/Summer2018--D20180905-T033113.raw", sonar_model='EK80')
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek80/D20170912-T234910.raw", sonar_model='EK80')

In [3]:
# EK60 files
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek60/ooi/CE02SHBP-MJ01C-07-ZPLSCB101_OOI-D20191201-T000000.raw", sonar_model='EK60')
# ed = ep.open_raw(TEST_DATA_FOLDER / "ek60/ncei-wcsd/Summer2017-D20170620-T011027.raw", sonar_model='EK60')

## Test the `Sonar/Beam_group1` group

### Read and process the CDL

In [4]:
target_group = "Sonar/Beam_group1"

In [5]:
conv_check = cc.ConventionCDL(target_group)

### Examine the target group in the converted EchoData object

In [6]:
ed[target_group]

<xarray.Dataset>
Dimensions:                        (channel: 2, ping_time: 2, beam: 4,
                                    range_sample: 191327)
Coordinates:
  * channel                        (channel) <U22 'WBT 714581-15 ES18' 'WBT 7...
  * ping_time                      (ping_time) datetime64[ns] 2019-12-14T08:1...
  * range_sample                   (range_sample) int64 0 1 2 ... 191325 191326
  * beam                           (beam) <U21 '1' '2' '3' '4'
Data variables: (12/21)
    frequency_nominal              (channel) float64 1.8e+04 3.33e+05
    beam_type                      (channel, ping_time) int64 1 1 1 1
    beamwidth_twoway_alongship     (channel, ping_time, beam) float64 10.18 ....
    beamwidth_twoway_athwartship   (channel, ping_time, beam) float64 10.33 ....
    beam_direction_x               (channel, ping_time, beam) float64 0.0 ......
    beam_direction_y               (channel, ping_time, beam) float64 0.0 ......
    ...                             ...
    sample_interval                (channel, ping_time) float64 nan ... nan
    transmit_power                 (channel, ping_time) float64 nan ... nan
    transmit_duration_nominal      (channel, ping_time) float32 nan ... nan
    slope                          (channel, ping_time) float64 nan 0.5 ... nan
    frequency_start                (channel, ping_time, beam) float64 nan ......
    frequency_end                  (channel, ping_time, beam) float64 nan ......
Attributes:
    beam_mode:              vertical
    conversion_equation_t:  type_3

In [7]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains complex backscatter data and other beam or channel-specific data.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

In [8]:
ed_group_ds = ed[target_group]

In [9]:
conv_check.set_ed_group_ds(ed_group_ds)

### Set the obligation and echopype_mods filters (optional)

In [10]:
conv_check.set_obligation(None) # "M", ['M', 'MA'], None, exclude=True

In [11]:
conv_check._get_obligation_vars()

array(['angle_alongship', 'angle_athwartship', 'angle_offset_alongship',
       'angle_offset_athwartship', 'angle_sensitivity_alongship',
       'angle_sensitivity_athwartship', 'backscatter_i', 'backscatter_r',
       'beam', 'beam_direction_x', 'beam_direction_y', 'beam_direction_z',
       'beam_stabilisation', 'beam_type', 'beamwidth_twoway_alongship',
       'beamwidth_twoway_athwartship', 'channel', 'equivalent_beam_angle',
       'frequency_end', 'frequency_nominal', 'frequency_start',
       'gain_correction', 'non_quantitative_processing', 'ping_time',
       'range_sample', 'sample_interval', 'sample_time_offset', 'slope',
       'transceiver_software_version', 'transmit_bandwidth',
       'transmit_duration_nominal', 'transmit_frequency_start',
       'transmit_frequency_stop', 'transmit_power', 'transmit_type'],
      dtype=object)

`echopype_mods` handling is not fully implemented yet

In [12]:
conv_check.set_echopype_mods(None) # None

In [13]:
conv_check.echopype_mods

### Run the tests

#### Presence of expected variables

In [14]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:
angle_alongship
angle_athwartship
beam_stabilisation
gain_correction
non_quantitative_processing
sample_time_offset
transmit_bandwidth
transmit_frequency_start
transmit_frequency_stop
transmit_type


#### Presence of unexpected variables

In [15]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [16]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:
transceiver_software_version: EchoData type: <U4, CDL type: object
frequency_end: EchoData type: float64, CDL type: int64
beam: EchoData type: <U21, CDL type: object
channel: EchoData type: <U22, CDL type: object
frequency_start: EchoData type: float64, CDL type: int64


In [17]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:
transceiver_software_version: EchoData type: <U4, CDL type: object
frequency_end: EchoData type: float64, CDL type: int64
frequency_start: EchoData type: float64, CDL type: int64


#### Variable dimensionality

In [18]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:


#### Attribute presence

In [19]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:
frequency_end: Missing EchoData attrs: {'valid_min', 'standard_name'}
frequency_start: Missing EchoData attrs: {'valid_min', 'standard_name'}
beam_type: Missing EchoData attrs: {'long_name'}


In [20]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

- This will be trickier b/c of values that are not defined as static
- **NOTE:** Currently excluding `valid_range` b/c it entails comparing lists or arrays, not scalars

In [21]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:


In [22]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:


## Test the `Platform` group

### Read and process the CDL

In [23]:
target_group = "Platform"

In [24]:
conv_check = cc.ConventionCDL(target_group)

### Examine the target group in the converted EchoData object

In [25]:
ed[target_group]

<xarray.Dataset>
Dimensions:                      (time1: 3, time2: 1, channel: 2)
Coordinates:
  * channel                      (channel) <U22 'WBT 714581-15 ES18' 'WBT 714...
  * time1                        (time1) datetime64[ns] 2019-12-14T08:13:44.6...
  * time2                        (time2) datetime64[ns] 2019-12-14T08:13:44
Data variables: (12/23)
    latitude                     (time1) float64 60.43 60.43 60.43
    longitude                    (time1) float64 5.528 5.528 5.528
    sentence_type                (time1) <U3 'GGA' 'GLL' 'RMC'
    pitch                        (time2) float64 -0.06
    roll                         (time2) float64 0.03
    vertical_offset              (time2) float64 0.0
    ...                           ...
    MRU_rotation_y               float64 nan
    MRU_rotation_z               float64 nan
    position_offset_x            float64 nan
    position_offset_y            float64 nan
    position_offset_z            float64 nan
    frequency_nominal            (channel) float64 1.8e+04 3.33e+05
Attributes:
    platform_name:       
    platform_type:       
    platform_code_ICES:

In [26]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains complex backscatter data and other beam or channel-specific data.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

In [27]:
ed[target_group].vertical_offset.min().values, ed[target_group].vertical_offset.max().values

(array(0.), array(0.))

In [28]:
ed[target_group].water_level.min().values, ed[target_group].water_level.max().values

(array(0.), array(0.))

In [29]:
if ed.sonar_model == "EK80":
    ed[target_group].drop_keel_offset.min().values, ed[target_group].drop_keel_offset.max().values

In [30]:
ed_group_ds = ed[target_group]

In [31]:
conv_check.set_ed_group_ds(ed_group_ds)

### Set the obligation and echopype_mods filters (optional)

In [32]:
conv_check.set_obligation(None) # "M", ['M', 'MA'], None, exclude=True

In [33]:
conv_check._get_obligation_vars()

array(['MRU_offset_x', 'MRU_offset_y', 'MRU_offset_z', 'MRU_rotation_x',
       'MRU_rotation_y', 'MRU_rotation_z', 'channel', 'distance',
       'drop_keel_offset', 'drop_keel_offset_is_manual',
       'frequency_nominal', 'heading', 'latitude', 'longitude', 'pitch',
       'position_offset_x', 'position_offset_y', 'position_offset_z',
       'roll', 'sentence_type', 'speed_ground', 'speed_relative',
       'tilt_x', 'tilt_y', 'time1', 'time2', 'transducer_offset_x',
       'transducer_offset_y', 'transducer_offset_z', 'vertical_offset',
       'water_level', 'water_level_draft_is_manual'], dtype=object)

`echopype_mods` handling is not fully implemented yet

In [34]:
conv_check.set_echopype_mods(None) # None

In [35]:
conv_check.echopype_mods

### Run the tests

#### Presence of expected variables

In [36]:
conv_check.test_vars_presence(test_type="expected")

****Expected variables not found in the EchoData object:
distance
heading
speed_ground
speed_relative
tilt_x
tilt_y


#### Presence of unexpected variables

In [37]:
conv_check.test_vars_presence(test_type="unexpected")

****EchoData variables not found in the CDL:


#### Variable data type

Use two types of data type comparisons: strict (specific) vs generalized.

In [38]:
conv_check.test_vars_datatype(dtype_strict=True)

****Variables with different data type from what is expected:
transducer_offset_z: EchoData type: float64, CDL type: float32
MRU_offset_z: EchoData type: float64, CDL type: float32
MRU_offset_y: EchoData type: float64, CDL type: float32
MRU_rotation_z: EchoData type: float64, CDL type: float32
transducer_offset_y: EchoData type: float64, CDL type: float32
position_offset_y: EchoData type: float64, CDL type: float32
position_offset_x: EchoData type: float64, CDL type: float32
vertical_offset: EchoData type: float64, CDL type: float32
roll: EchoData type: float64, CDL type: float32
transducer_offset_x: EchoData type: float64, CDL type: float32
position_offset_z: EchoData type: float64, CDL type: float32
MRU_rotation_y: EchoData type: float64, CDL type: float32
MRU_offset_x: EchoData type: float64, CDL type: float32
water_level: EchoData type: float64, CDL type: float32
MRU_rotation_x: EchoData type: float64, CDL type: float32
sentence_type: EchoData type: <U3, CDL type: object
channel: E

In [39]:
conv_check.test_vars_datatype(dtype_strict=False)

****Variables with different data type from what is expected:
sentence_type: EchoData type: <U3, CDL type: object


#### Variable dimensionality

In [40]:
conv_check.test_vars_dimensionality()

****EchoData variables with dimensionality different from the CDL:


#### Attribute presence

In [41]:
conv_check.test_attrs_presence()

****Variable or global missing convention attributes:
drop_keel_offset_is_manual: Missing EchoData attrs: {'comment'}
drop_keel_offset: Missing EchoData attrs: {'comment'}
water_level_draft_is_manual: Missing EchoData attrs: {'comment'}


In [42]:
conv_check.test_attrs_presence(global_attrs=True)

****Variable or global missing convention attributes:


#### Attribute values

- This will be trickier b/c of values that are not defined as static
- **NOTE:** Currently excluding `valid_range` b/c it entails comparing lists or arrays, not scalars

In [43]:
conv_check.test_attrs_value()

****Variable or global attribute with different values:


In [44]:
conv_check.test_attrs_value(global_attrs=True)

****Variable or global attribute with different values:
